Compare and visualize performance of a domestic asset vs foreign
===

In [1]:
import eikon as ek
import pandas as pd
import plotly.offline as offline
import plotly.graph_objs as go

In [2]:
# eikon id is generated from Eiokon API proxy
ek.set_app_id('your_app_id_here')
# set notebook mode to allow charts to be generated inline
offline.offline.init_notebook_mode(connected=True)

Define variables


In [3]:
# the two instruments whose performance is to be compared
instr1 = 'TD.TO';
instr2 = 'BARC.L'
# the date range for comparison
start_date = '2015-01-01'
end_date = '2017-01-01'

Get the primary currency in which these instruments trade


In [4]:
data,err = ek.get_data([instr1, instr2], "TR.CompanyMarketCap.Currency")
# display the trading currencies
data.head()

,Instrument,Currency
0,TD.TO,CAD
1,BARC.L,GBP


Create a currency cross rate RIC with this information, and get timeseries data for this pair

In [5]:
# cross rate ric = curr1curr2=X. It is a calculated metric
crossRIC = data['Currency'][1] + data['Currency'][0] + '=X'
curr = ek.get_timeseries([crossRIC], fields='CLOSE', start_date=start_date, end_date=end_date)
curr.head()

GBPCAD=X,CLOSE
Date,
2015-01-01,1.8054
2015-01-02,1.8057
2015-01-05,1.7935
2015-01-06,1.7930
2015-01-07,1.7849


Get timeseries data for both instruments in the date range

In [6]:
perf1 = ek.get_timeseries(instr1, fields='CLOSE', start_date=start_date, end_date=end_date)
perf2 = ek.get_timeseries(instr2, fields='CLOSE', start_date=start_date, end_date=end_date)

Scale the performance data for foreign asset with curreny performance

In [7]:
# create a single DataFrame with both data series (useful if using cufflinks to plot instead of plotly)
perf = pd.concat([perf1, perf2 * curr], axis=1)
perf.columns = [instr1, instr2]
# drop invalid entries from dataframe
perf = perf.dropna()
perf.head()

,TD.TO,BARC.L
Date,,
2015-01-02,55.22,439.055955
2015-01-05,53.94,420.934450
2015-01-06,53.06,413.017550
2015-01-07,52.89,411.597940
2015-01-08,53.23,423.039430


Rebase the data

In [8]:
# rebase both timeseries to 100% at start date
perf_rebase = perf * 100 / (perf[instr1][0], perf[instr2][0])
perf_rebase.head()

,TD.TO,BARC.L
Date,,
2015-01-02,100.000000,100.000000
2015-01-05,97.681999,95.872621
2015-01-06,96.088374,94.069456
2015-01-07,95.780514,93.746124
2015-01-08,96.396233,96.352054


Create the performance line chart

In [9]:
# using plotly for charting
# create the line chart for both instruments
perfFig = [{
    'x': perf_rebase.index,
    'y': perf_rebase[col],
    'name': col,
    'line': dict( width=1)
}  for col in perf_rebase.columns]


Create the currency spread chart

In [10]:
# create the currency spread chart
dPv = curr['CLOSE'] - curr['CLOSE'][0];
dNv = dPv.copy();
dPv[dPv < 0] = 0.0;
dNv[dNv >= 0] = 0.0;

spreadPv = go.Scatter(
    x=perf_rebase.index,
    y=dPv,
    fill='tozeroy',
    line=dict(
        color='green',
        width=0
    ),
    yaxis='y2',
    name='currency',
    showlegend=False,
);

spreadNv = go.Scatter(
    x=perf_rebase.index,
    y=dNv,
    fill='tozeroy',
    line=dict(
        color='red',
        width=0
    ),
    yaxis='y2',
    name='currency',
    showlegend=False,
);

Plot these charts using plotly

In [11]:
perfFig.append(spreadPv);
perfFig.append(spreadNv);

layout = go.Layout(
    title = 'Performance and Currency over time',
    yaxis1=dict(
        domain=[0.3, 1.0]
    ),    
    yaxis2=dict(
        domain=[0.0, 0.25]
    ),
    xaxis1=dict(
        zerolinecolor='red'
    )
);

# using plotly for charting
fig = go.Figure(data=perfFig, layout=layout)
offline.iplot(fig)

Simplified Code
===
This code achieves the same chart, by utilizing the currency transformation at server side

In [12]:
# get and rebase the data
response, error = ek.get_data(instruments=[instr1, instr2], fields=['TR.ClosePrice.Date', 'TR.ClosePrice.Value'], parameters={'Curn':'CAD','SDate':start_date,'EDate':end_date,'Frq':'D'})
df = response.pivot_table(values='Close Price', index=['Date', 'Instrument']).unstack('Instrument').dropna()
rebased = df.apply(lambda series: series/series[0]*100)

# plot it in a chart
offline.iplot([{
    'x': rebased.index,
    'y': rebased[column],
    'name': column[1]
}  for column in rebased.columns])